In [20]:
import pandas as pd
import numpy as np

In [21]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

In [22]:
df_turnover = pd.read_csv('/Users/sahityaraipura/Desktop/turnover.csv')

In [23]:
df_turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [24]:
df_turnover.shape

(14999, 10)

In [25]:
def train_cats(df):
    for n, c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype('category').cat.as_ordered()

In [26]:
train_cats(df_turnover)

In [29]:
df_turnover.salary.cat.codes

0        1
1        2
2        2
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
14969    2
14970    2
14971    2
14972    2
14973    2
14974    2
14975    2
14976    2
14977    2
14978    2
14979    2
14980    0
14981    1
14982    2
14983    2
14984    2
14985    2
14986    1
14987    1
14988    1
14989    1
14990    1
14991    1
14992    1
14993    1
14994    1
14995    1
14996    1
14997    1
14998    1
Length: 14999, dtype: int8

In [30]:
df_turnover.sales.cat.codes

0        7
1        7
2        7
3        7
4        7
5        7
6        7
7        7
8        7
9        7
10       7
11       7
12       7
13       7
14       7
15       7
16       7
17       7
18       7
19       7
20       7
21       7
22       7
23       7
24       7
25       7
26       7
27       7
28       2
29       2
        ..
14969    7
14970    7
14971    7
14972    2
14973    2
14974    2
14975    3
14976    3
14977    3
14978    3
14979    9
14980    9
14981    9
14982    9
14983    9
14984    9
14985    9
14986    9
14987    9
14988    9
14989    9
14990    8
14991    8
14992    8
14993    8
14994    8
14995    8
14996    8
14997    8
14998    8
Length: 14999, dtype: int8

In [31]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name +'_na'] = pd.isnull(col)
#         print(name)
        df[name] = df[name].fillna(df[name].median())

#If the column in dataframe is not numeric, then for the purpose of feeding data to our ML algorithms, we numeracailize it

def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and (max_n_cat is None or col.unique() > max_n_cat):
        df[name] = col.cat.codes + 1


def proc_df(df, y_fld, skip_flds = None, do_scale= False, preproc_fn = None, max_n_cat = None, subset = None):
    if not skip_flds:
        skip_flds = []
    if subset:
        df = get_sample(df, subset)
    df = df.copy()
    if preproc_fn:
        preproc_fn(df)
    y = df[y_fld].values
    df.drop(skip_flds + [y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df,c,n)
    if do_scale:
        scale_vars(df)
    for n,c in df.items():
        numericalize(df, c, n, max_n_cat)
    res = [pd.get_dummies(df, dummy_na= True),y]
    if not do_scale:
        return res
    return res + [mapper]

In [32]:
df_new, y = proc_df(df_turnover, 'left')

In [33]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [39]:
m = RandomForestClassifier()
m.fit(df_new,y)
m.score(df_new,y)*100


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


99.84665644376291

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_new, df_turnover.left,test_size = 0.2)

In [38]:
rand = RandomForestClassifier()
rand.fit(x_train, y_train)
print(rand.score(x_test, y_test)*100)
pred = rand.predict(x_test)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


98.93333333333332


In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[2269,    8],
       [  23,  700]])